<a href="https://colab.research.google.com/github/compartia/AI-tecture/blob/dev/miro_openai_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Miro and Chat GPT

 status: **DRAFT**

 **by:** Artem Zaborskiy


 system that integrates AI-generated content with Miro (https://miro.com/), a collaborative online whiteboard platform. It specifically handles the part of the workflow where an AI-generated response is posted back to the Miro board, enhancing the interactive and dynamic capabilities of the board by incorporating AI insights directly into the collaborative workspace.



### TODOs:

1. ~~sort nodes to process, proritize, what propmps to process first~~
1. cache processing results,
1. ~~check dates or checksums~~
1. check for self-connected nodes
1. check for prompt-prompt connectors
1. do not reload the entire board after widget created: add new widget and its connector to dataframes
1. ~~create output shape if propmpt node has no output connectors~~
1. Update shapes chache data tables after AI answer is received

## Prepare OPEN AI client

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
!pip install --upgrade openai typing-extensions

In [ ]:
import openai
from google.colab import userdata

ai_client = openai.OpenAI(api_key=userdata.get('OP_API')  )

In [ ]:
BOARD_ID = 'uXjVN61x8Pg='
DEBUG_OPENAI = False

### Test open AI api calls

In [ ]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )

  # print(completion.choices[0].message)
  display(HTML(completion.choices[0].message.content))

In [ ]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )
  #
  # print(completion.choices[0].message)
  display(HTML(completion.choices[0].message.content))

In [ ]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )

  display(HTML(completion.choices[0].message.content))

## OpenAI API tools

In [ ]:
def ask_ai(prompt, argument_context):

  completion = ai_client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": f"{prompt}"},
      # {"role": "system", "content": f"{prompt}. Your answer is in a form of list of <p> HTML tags and has NO newline '\n' symbols."},
      {"role": "user", "content": argument_context}
    ]
  )
  print(completion.choices)
  return completion.choices[0].message.content

if DEBUG_OPENAI or True:
  r = ask_ai('you are a female philosopher. Answer in 2 words.', 'what is the meaning of life?')
  print(r)
  display(HTML(r))
  # display(HTML(r))

In [ ]:
if DEBUG_OPENAI or True:
  r = ask_ai('you are a male philosopher. Answer in 2 words.', 'what is the meaning of life?')
  print(r)
  display(HTML(r))
  # display(HTML(r))

# Miro client
https://developers.miro.com/reference/api-reference

In [ ]:
import requests
import pandas as pd


class MiroClient:
  def __init__(self, token, board_id):
    self.board_id = board_id
    self.token = token
    self.base_url = f"https://api.miro.com/v2/boards/{board_id}"

    self.headers = {
        "accept": "application/json",
        "authorization": f"Bearer {self.token}"
    }
    self.connectors_df:pd.DataFrame or None = None
    self.shapes_df:pd.DataFrame or None = None


  def get_items(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/items?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/items?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response

  def get_connectors(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/connectors?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/connectors?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response


  def get_all_pages(self, page_method):

    cursor = None

    max_steps = 30
    step = 0
    while True:
      step += 1
      response = page_method(cursor)
      rj = response.json()
      yield rj
      cursor = rj.get('cursor')
      if cursor is None or step > max_steps:
        break


miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)


## Getting all connectors

In [ ]:
def decode_connectors(page:dict):
  # print(page)
  for i in page['data']:
    yield i['id'], i.get('startItem', {}).get('id'), i.get('endItem', {}).get('id')


##-----
## test it
one_page_con = miro_client.get_connectors(limit=10).json()
x = [i for i in decode_connectors(one_page_con)]
x

In [ ]:
def get_all_connectors(self):
  for page in self.get_all_pages(self.get_connectors):
    for i in decode_connectors(page):
      yield i


def all_connectors_as_df(self):
  all_connectors_iter = self.get_all_connectors()
  self.connectors_df = pd.DataFrame(all_connectors_iter, columns=['id','id_from', 'id_to'])
  self.connectors_df = self.connectors_df.set_index('id')

  return self.connectors_df


MiroClient.get_all_connectors = get_all_connectors
MiroClient.all_connectors_as_df = all_connectors_as_df



##-----
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

miro_client.all_connectors_as_df()
miro_client.connectors_df


## Getting all shapes (widgets)

In [ ]:
def decode_items(page:dict):

  for i in page['data']:
    # print(type(i))
    if type(i) == dict :
      yield i['id'], i['type'], i['data'].get('shape', '_undefined_'), i['data'].get('content'), i['modifiedAt'], i['geometry'], i['position']

    else:
      print('NOT A DICT!!', type(i), i)
      decode_items(i)



def get_all_items(self):
  for page in self.get_all_pages(self.get_items):
    # print( page.get('cursor'), page.get('size'), page.get('total') )
    for i in decode_items(page):
      yield i
      # print(i)


MiroClient.get_all_items = get_all_items

In [ ]:

def all_items_as_df(miro_client):
  items_iter = miro_client.get_all_items()

  df = pd.DataFrame(items_iter, columns=['id','type', 'shape', "contents", 'modifiedAt', 'geometry', 'position'])
  df = df.set_index('id')
  df['modifiedAt_date'] = pd.to_datetime(df['modifiedAt'])

  miro_client.shapes_df = df
  return miro_client.shapes_df


MiroClient.all_items_as_df = all_items_as_df
# --------

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
shapes_df = miro_client.all_items_as_df()

In [ ]:
miro_client.shapes_df

### Update board item

In [ ]:
def get_shape_info(self, shape_id):
  url = f"{self.base_url}/shapes/{shape_id}"
  response = requests.get(url, headers=self.headers)
  print(response.text)

def get_stiky_note_info(self, shape_id):
  url = f"{self.base_url}/sticky_notes/{shape_id}"
  response = requests.get(url, headers=self.headers)
  print(response.text)


def update_item(self, shape_id, item_type, content=None, shape=None, color = None):

  url = None

  if content is None:
    payload = {}
  else:
    payload = {
        "data": {
            "content": content
        }
    }


  if color is not None:
    payload['style']={}
    payload['style']['fillColor'] = color

    if item_type!='sticky_note':
      payload['style']['fillOpacity'] = "1.0"


  if item_type=='sticky_note':
    url = f"{self.base_url}/sticky_notes/{shape_id}"
  elif item_type=='shape':
    url = f"{self.base_url}/shapes/{shape_id}"
    if shape is not None:
      payload['data']['shape'] = shape

  print(payload)


  response = requests.patch(url, json=payload, headers=self.headers)
  print(response.text)
  return response



MiroClient.update_item = update_item
MiroClient.get_shape_info = get_shape_info
MiroClient.get_stiky_note_info = get_stiky_note_info

#-----

# miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

# if False:

  # miro_client.get_stiky_note_info('3458764578213963472')
# test_id = '3458764578216605194'
# miro_client.update_item( test_id, item_type=shapes_df.at[test_id, 'type'], content=completion.choices[0].message.content,  color='#ff0000')

In [ ]:
def create_connector(self, form, to, text):
  import requests

  url = f"{self.base_url}/connectors"

  payload = {
      "startItem": {
          "id": form,
          "snapTo": "right"
      },
      "endItem": {
          "id": to,
          "snapTo": "auto"
      },
      "style": { "endStrokeCap": "arrow" },
      "shape": "curved",
      "captions": [{ "content": text }]
  }

  response = requests.post(url, json=payload, headers=self.headers)
  return response


def create_stiky_note(self, contents, pos):

  url = f"{self.base_url}/sticky_notes"

  payload = {
    "data": {
        "content": contents,
        "shape": "rectangle"
    },
    "style": {
        "fillColor": "pink",
        "textAlign": "left"
    },
    "position": {
        "x": pos[0],
        "y": pos[1]
    },
    "geometry": {
        "height": pos[2]
        # "width": pos[3]
    }
  }

  response = requests.post(url, json=payload, headers=self.headers)
  return response


MiroClient.create_stiky_note = create_stiky_note
MiroClient.create_connector = create_connector


# --------

def get_out_items(self, node_id):
  _links = self.connectors_df[ self.connectors_df['id_from']==node_id]
  return self.shapes_df.loc [ _links['id_to'] ]

def get_in_items(self, node_id):
  _links = self.connectors_df[ self.connectors_df['id_to']==node_id]
  return self.shapes_df.loc [_links['id_from'] ]

MiroClient.get_out_items=get_out_items
MiroClient.get_in_items=get_in_items

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
# response = miro_client.create_stiky_note( "TESTCREATE", (20,20,400,400) )
# response.json()['id']

# Sorting nodes (Tokpologically) witn networkx

In [ ]:
# !pip install networkx

In [ ]:
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

miro_client.all_connectors_as_df()
miro_client.all_items_as_df()

## Perform a topological sort on the condensed DAG
- https://arxiv.org/pdf/2110.07580.pdf

- Step 1: Identify SCCs with NetworkX `networkx.strongly_connected_components(G)`.
- Step 2: Contract (condense) each SCC into a single node, reducing the original graph into a DAG where each node represents an SCC. This is  called the **"condensation"** of the graph.
- Step 3: Perform a topological sort on the condensed DAG. This gives an order of the SCCs, which can serve as a coarse **hierarchical ordering** of the original graph's components.

In [ ]:
import networkx as nx

# Create a directed graph
G = nx.DiGraph()

for i, e in miro_client.connectors_df.iterrows():
  # print (e)
  G.add_edge(e['id_from'], e['id_to'])

sccs = list(nx.strongly_connected_components(G))

# Step 2: Create a condensed graph of SCCs
condensed_graph = nx.condensation(G, sccs)
for i, scc in enumerate(sccs):
    # The nodes of the original graph in the current SCC
    original_nodes = condensed_graph.nodes[i]['members']
    if len(original_nodes)>1:
      print(f"SCC {i}: Contains original nodes {original_nodes}")



# Step 3: Perform a topological sort on the condensed graph
topological_order_sccs = list(nx.topological_sort(condensed_graph))

# Printing the order of SCCs


In [ ]:
# TEST
scc_index = 0  # For example, to get the first SCC in the topological order
# Retrieve the node set (members) of the original graph that form this SCC
original_nodes_in_scc = condensed_graph.nodes[topological_order_sccs[scc_index]]['members']
print(f"Original nodes in SCC {scc_index}: {original_nodes_in_scc}")

In [ ]:
#Collect IDs

In [ ]:

ordered_nodes = []
for i in topological_order_sccs:
  original_nodes_in_scc = condensed_graph.nodes[topological_order_sccs[i]]['members']
  for n in original_nodes_in_scc:
    ordered_nodes.append(n)
    miro_client.shapes_df.at[n, 'order']=i

ordered_shapes_df = miro_client.shapes_df.loc[ordered_nodes].sort_values(by='order')
ordered_shapes_df.tail(4)

# Processing the board
Updating Miro board widgets with an answers generated by an AI model. Here's a breakdown of its functionality:






### Collect shapes to process
1. get all prompts
2. get **linked** nodes
3. **invalidate** nodes and prompts that updated later than answer nodes
4. **prioritize** nodes
5. **Update Widget**:  the target widget(s) is updated with AI-generated answer. This update is performed by calling the `update_item` method of the `miro_client` object

## 1. get all prompts

In [ ]:
prompts_df = ordered_shapes_df[ordered_shapes_df['shape']=='parallelogram'].sort_values(by='order')
prompts_df

#### create missing outputs

In [ ]:
default_contents='waiting for AI to answer ....'

In [ ]:


def _build_output_shape(shape_id, contents = default_contents):
  shapes_df = miro_client.shapes_df
  shape = shapes_df.loc [ shape_id ]

  pos = shape['position']
  geo = shape['geometry']

  size = max(geo['width'], geo['height'])

  response = miro_client.create_stiky_note( contents, (pos['x'] +  geo['width'] + size, pos['y'], size, size) )
  a_id = response.json()['id']

  response = miro_client.create_connector( shape_id,  a_id, 'AI answer' )

  return a_id

# test------
# build_output_shape('3458764578079690837', "TESTCREATE")


_needs_reload = False
for prompt_id in prompts_df.index:
  out_links = miro_client.connectors_df[ miro_client.connectors_df['id_from']==prompt_id]
  if len(out_links) == 0:
    out_shape_id = _build_output_shape(prompt_id, contents=default_contents)
    _needs_reload = True

if _needs_reload:
  #TODO: OPTIMIZE!!
  print ('reloading board')
  miro_client.all_items_as_df()
  miro_client.all_connectors_as_df()

del _needs_reload

### TODO: descendants = nx.descendants(G, start_node)


- Identify which prompts (or shapes) need to be updated based on the **modification dates** of incoming and outgoing items. If a prompt is older than any item feeding into it, it is considered outdated.

- Determine the impact of these updates, using a directed graph to find all **descendants** of the outdated prompts. This implies that any changes to a prompt potentially invalidate all items that are downstream from it in the workflow or dependency graph.

- Compile a list of shapes that need to be updated

In [ ]:
#  a set to keep track of IDs that need to be updated
invalidated_ids = set()

# Iterate over each prompt ID
for prompt_id in prompts_df.index:

  # Retrieve outgoing/incoming items related to the current prompt ID
  out_items =  miro_client.get_out_items(prompt_id)
  in_items =  miro_client.get_in_items(prompt_id)

  # DATES
  # Find the maximum/min modification date among incoming/outgoing items
  in_date = in_items.modifiedAt_date.max()
  out_date = out_items.modifiedAt_date.min()

  # Get the modification date of the current prompt
  prompt_date = miro_client.shapes_df.at[prompt_id, 'modifiedAt_date']

  # Check if the current prompt is older than any of the incoming items
  if (prompt_date < in_date) or (prompt_date > out_date):
    # If so, find all descendants of the prompt ID in the graph (items affected by changes)
    descendants = nx.descendants(G, prompt_id)
    for d in descendants:
      # Add the descendant ID to the set of invalidated IDs
      invalidated_ids.add(d)


    # This line is commented out; it seems it was an alternative way to add out_items to invalidated_ids

# Select shapes from a DataFrame that match the invalidated IDs
shapes_to_update = ordered_shapes_df.loc[invalidated_ids]
# Filter out shapes that are not 'parallelogram' from the shapes to update
# shapes_to_update = shapes_to_update[shapes_to_update['shape'] != 'parallelogram']
shapes_to_update

#### Render invalidated shapes as red

In [ ]:
def get_color_for_state(out_shape_id, state):
  item_type = miro_client.shapes_df.at[out_shape_id, 'type']

  if state==0:
    if item_type=='sticky_note':
      color = 'red'
    else:
      color = '#ff5566'

  if state==1:
    if item_type=='sticky_note':
      color = 'light_green'
    else:
      color = '#ccffdd'

  return color

In [ ]:
_outputs = shapes_to_update[shapes_to_update['shape'] != 'parallelogram']
for out_shape_id, item in _outputs.iterrows():
  color = get_color_for_state(out_shape_id, 0)
  res = miro_client.update_item( out_shape_id, item_type=item['type'], content=f"<p><i>order: {int(item['order'])}</i></p>   {default_contents}", color = color)
del _outputs

#### Calling AI for answers

In [ ]:
def get_output_shape_id(prompt_id):
  out_links = miro_client.connectors_df[ miro_client.connectors_df['id_from']==prompt_id]
  if len(out_links) > 0:
    _items = miro_client.shapes_df.loc[out_links['id_to'] ]
    return _items.iloc[0].name

def get_incoming_text(prompt_id):

  incoming_items = miro_client.get_in_items(prompt_id).sort_values(by='position', key=lambda x: x.map(lambda d: d['y']), ascending=True)
  concatenated_text = '\n\n'.join(incoming_items.contents)

  return concatenated_text

In [ ]:
def build_answer(prompt_id):

  connectors_df = miro_client.connectors_df
  shapes_df = miro_client.shapes_df
  incoming_links = connectors_df[ connectors_df['id_to']==prompt_id]
  out_shape_id = get_output_shape_id(prompt_id)



  if out_shape_id in invalidated_ids:

    _propmt_text = shapes_df.loc[prompt_id]['contents']
    in_text = get_incoming_text(prompt_id)

    _item_type = shapes_df.at[out_shape_id, 'type']
    answer_text = ask_ai(_propmt_text, in_text)

    try:
      display(HTML(answer_text))
    except:
      #it cannot display malformed HTML
      print('cannot render html', answer_text)

    color = get_color_for_state(out_shape_id, 1)

    #update cache with AI answer
    shapes_df.at[out_shape_id, 'contents'] = answer_text
    res = miro_client.update_item( out_shape_id, item_type=_item_type, content=answer_text,  shape='round_rectangle', color = color)

  return answer_text



# propmpts are sorted
for prompt_id in prompts_df.index:
  print('-'*40)
  print(prompt_id, 'processing')
  answer_text = build_answer(prompt_id)



